# Airline Review Analysis - In Progress


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data Gathering and Cleaning

Initially, I thought of scrapping the [skytrax](vhttps://www.airlinequality.com/) website, which seemed pretty easy. Before doing that however, a quick search showed me that someone has already done that and the [data uploaded on Kaggle](https://www.kaggle.com/efehandanisman/skytrax-airline-reviews) is fairly recent.  
  
Actually, the fact that it does not include reviews of the last months may be a good thing. As I was browsing skytrax I noticed many recent negative reviews of cancellations, new no-refund policies and lockdown measures, that were caused by Covid-19. Hopefully, this does not reflect the reality of air traveling and we avoid using it in our analysis. 

In [2]:
# We drop the aircraft column since it has many NaN values and is not important for our analysis
df = pd.read_excel('datasets/capstone_airline_reviews3.xlsx').drop(['aircraft'], axis=1)

# Many fields will remain empty, but depending on our use case we will accordingly deal with them
df = df.dropna(how='all')

In [3]:
display(df)

,airline,overall,author,review_date,customer_review,traveller_type,cabin,route,date_flown,seat_comfort,cabin_service,food_bev,entertainment,ground_service,value_for_money,recommended
1,Turkish Airlines,7.0,Christopher Hackley,8th May 2019,âœ… Trip Verified | London to Izmir via Istanb...,Business,Economy Class,London to Izmir via Istanbul,2019-05-01 00:00:00,4.0,5.0,4.0,4.0,2.0,4.0,yes
3,Turkish Airlines,2.0,Adriana Pisoi,7th May 2019,âœ… Trip Verified | Istanbul to Bucharest. We ...,Family Leisure,Economy Class,Istanbul to Bucharest,2019-05-01 00:00:00,4.0,1.0,1.0,1.0,1.0,1.0,no
5,Turkish Airlines,3.0,M Galerko,7th May 2019,âœ… Trip Verified | Rome to Prishtina via Ista...,Business,Economy Class,Rome to Prishtina via Istanbul,2019-05-01 00:00:00,1.0,4.0,1.0,3.0,1.0,2.0,no
7,Turkish Airlines,10.0,Zeshan Shah,6th May 2019,âœ… Trip Verified | Flew on Turkish Airlines I...,Solo Leisure,Economy Class,Washington Dulles to Karachi,April 2019,4.0,5.0,5.0,5.0,5.0,5.0,yes
9,Turkish Airlines,1.0,Pooja Jain,6th May 2019,âœ… Trip Verified | Mumbai to Dublin via Istan...,Solo Leisure,Economy Class,Mumbai to Dublin via Istanbul,2019-05-01 00:00:00,1.0,1.0,1.0,1.0,1.0,1.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131886,Ukraine International,NaN,Yevheniy Kahanovych,21st January 2007,Two domestic flights between Kiev and Lviv las...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
131888,Ukraine International,NaN,Helen Lee,15th December 2006,I had a good flight from London to Kiev with U...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
131890,Ukraine International,NaN,Andriy Yesypenko,19th May 2006,Kiev - London (Gatwick) in business class (in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
131892,Ukraine International,NaN,Volodya Bilotkach,29th April 2006,Several flights - KBP to AMS (3 times one way)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no


This is a first read for our dataset and we have a lot of cleaning up to do. Depending on what data we need for our analysis we must deal with the remaining `NaN` values.  
Also on the `customer_review` field we can see some unicode characters that must be dealt with.

## Most Important Rating

Each review has a number of ratings:
* Food & Beverages
* Inflight Entertainment
* Seat Comfort
* Value For Money
* Cabin Staff Service
* Ground Service
* Wifi & Connectivity

THe ratings have a value from 1 (being the worst) to 5 (being the best). I would like to find which one of the ratings is the most correlated with **if he would recommend the flight**.  
  
Why I consider the *recommend* value as the best reflection of passenger satisfaction?
If a passenger recommends an airline then this means two things. Firstly, that he would promote to a friend/family the airline. Secondly, in future air traveling is probable that hew would again choose the same airline if possible. From a business perspective I believe this makes the *recommend* metric important.  

### Missing Values

As expected many of the reviews have missing values. The quick approach is to drop all the rows with missing values on the columns we need. We will do that and check how many rows we lose.

In [10]:
# Select the columns we need from the df
rate_importance_df = df.iloc[:,-7:].copy()

# We drop the rows that have a NaN value on the recommended column
rate_importance_df = rate_importance_df[rate_importance_df['recommended'].notna()]

print(f"Rows including NaNs: {len(rate_importance_df)}")

rate_importance_df = rate_importance_df.dropna()

print(f"Rows without NaNs: {len(rate_importance_df)}")

Rows including NaNs: 64440
Rows without NaNs: 24596


We lose about 40.000 rows but we still have ~25.000 rows. At least initially I feel that the remaining rows are enough and no other strategy is followed.

## Correlation

We will now calculate the correlation of each rating with if the *recommendation* value.  
  
Our results will not have a simple interpretation, meaning we must take into account that [**correlation does not imply causation**](https://www.tylervigen.com/spurious-correlations).  
  
*Hypothetical scenario:  
Let's say we found a great correlation (r >0.9) between recommend and inflight entertainment. One might suggest we should make our flights more entertaining, so we install TVs, music, etc. but after some months we do not have the expected increase on ratings.  
With further analysis we discover that the duration of the flight is highly correlated with the inflight entertainment and consequently with the recommend value.  
This means that long haul flights make the passengers more easily bored and tired and we cannot actually do much about this specific issue since [airplanes will not get much faster](https://aviation.stackexchange.com/a/31386).

In [12]:
# Change the yes/no values on recommend column to 1/0
rate_importance_df.recommended = rate_importance_df.recommended.map(dict(yes=1, no=0))



,seat_comfort,cabin_service,food_bev,entertainment,ground_service,value_for_money,recommended
1,4.0,5.0,4.0,4.0,2.0,4.0,1
3,4.0,1.0,1.0,1.0,1.0,1.0,0
5,1.0,4.0,1.0,3.0,1.0,2.0,0
7,4.0,5.0,5.0,5.0,5.0,5.0,1
9,1.0,1.0,1.0,1.0,1.0,1.0,0
...,...,...,...,...,...,...,...
131696,1.0,1.0,1.0,1.0,1.0,1.0,0
131698,1.0,2.0,1.0,1.0,1.0,1.0,0
131702,5.0,5.0,5.0,4.0,4.0,4.0,1
131704,1.0,1.0,1.0,1.0,1.0,1.0,0
